# Save graph and variables in tensorflow

我们在用tensorflow训练了一个模型之后，经常需要保存这个模型并在之后复用。由于tensorflow使用graph来描述计算结构，而实际计算在session中进行，所以tensorflow保存模型的时候是将graph和session中的variable分开保存。

## 一、tf.train.Saver

该类是tf中用来保存varibale的常用类，它需要在一个graph中被初始化，而调用其save()方法则必须是在一个session中。Saver将所有的变量保存到一个checkpoint文件中，而且提供了一种方法，使得你在训练过程中可以保存不同时间的checkpoint文件，并能自动给其按照数字编码。

`该类调用save()方法的时候会隐含地调用tf.train.Saver.export_meta_graph(),将这个session的graph保存起来`

### 1. tf.train.Saver.save(sess, save_path, global_step=None, latest_filename=None, meta_graph_suffix='meta', write_meta_graph=True, write_state=True)

这个函数可以将一个session中的变量保存到checkpoints文件中，同时它可以将该session的graph保存起来。


In [1]:
import tensorflow as tf

In [2]:
W = tf.Variable([[1,2],[3,4]],dtype = tf.float32)

b = tf.Variable([1,2],dtype=tf.float32)

init = tf.global_variables_initializer()

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.save(sess,"checks/mymodel")
    

我们可以发现saver在checks文件夹下生成了四个文件：

<img src="checkpoint_file.png">

其中checkpoint文件是tensorflow用来追踪生成的这些保存variable文件的，而mymodel.data-00000-of-00001是真实保存variable的文件，而mymodel.meta是保存graph的文件。

### 2. tf.train.Saver.restore(sess, save_path)

restore()方法用于从保存的checkpoint文件中恢复varible。使用这个方法有两点要注意：
1. 该方法需要在一个session中调用，且这个session要有一个被加载的graph。也就是说，这个graph中要有需要被恢复的variable的信息。
2. 需要被恢复的variable不需要被初始化，因为restore本身就能对这些variable进行初始化。
2. save_path这个参数被传入的值应该是之前save()方法返回的值。我们在上一步中可以看到，虽然我们将save()方法中的save_path参数设定为"checks/mymodel",但是tensorflow将其保存为mymodel.data-00000-of-00001，但是如果我们将其传入restore中是会报错的，也就是说，我们还是应该传入"checks/mymodel",当然有时候会加上一些表示不同step的后缀。


In [12]:
with tf.Graph().as_default():  #之所以加上这一行，是因为在jupyter notebook中所有的代码都是逻辑连续的，这样可以新起一个graph，否则
    #W的name就会变成"Varibale_3:0",因为在之前的graph中已经定义了一次W和b
    W = tf.Variable([[0,0],[0,0]],dtype = tf.float32)
    b = tf.Variable([0,0],dtype=tf.float32)

    new_saver = tf.train.Saver()
    with tf.Session() as sess:
        new_saver.restore(sess,"checks/mymodel")
        print("W:",sess.run(W))
        print("b",sess.run(b))

W: [[ 1.  2.]
 [ 3.  4.]]
b [ 1.  2.]


### 3.tf.train.Saver.restore()是如何恢复varible的数值的？

restore()方法主要是根据当前这个session的graph中变量的op.name来查找的，首先对checkpoint文件中储存的每一个变量，去看它的key，然后对当前graph中的varible，查找op.name等于key的变量



```python
with tf.Graph().as_default():
    with tf.Session() as sess:
        new_saver =tf.train.import_meta_graph('checks/mymodel.meta')
        new_saver.restore(sess,"checks/mymodel")
        print("W:",sess.run(W))
        print("b:",sess.run(b))
```
上述代码会报错，因为虽然restore()可以恢复导入的'checks/mymodel.meta'中的graph的variable,但是我们并没有将这些变量显示地赋值给python变量，所以`print("W:",sess.run(W))`是会出错的。

我们要使用之前graph中的variable，必须先获得这个variable，然后显式地将其赋值给一个python变量。

在tensorflow中有多种方法可以获得一个变量：
1. tf.Graph.get_tensor_by_name
2. 将变量在原graph中加入到一个collection中，然后在恢复变量时从这个collection中获取

In [11]:
# 利用get_tensor_by_name
with tf.Graph().as_default():
    with tf.Session() as sess:
        new_saver =tf.train.import_meta_graph('checks/mymodel.meta')
        new_saver.restore(sess,"checks/mymodel")
        
        #因为在原来的graph中，W是一个Variable对象，由于我们没有指定name参数，因此W的name就是Variable:0
        W = tf.get_default_graph().get_tensor_by_name("Variable:0")
        print("W:",sess.run(W))



W: [[ 1.  2.]
 [ 3.  4.]]


从tf.get_collection()中来获取变量：

```python
W = tf.Variable([[1,2],[3,4]],dtype = tf.float32)

b = tf.Variable([1,2],dtype=tf.float32)

init = tf.global_variables_initializer()

tf.add_to_collection("weights", W)
tf.add_to_collection("bias",b)

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.save(sess,"checks/mymodel")
```

之后我们可以重新导入graph并恢复：

```python
with tf.Graph().as_default():
    with tf.Session() as sess:
        new_saver =tf.train.import_meta_graph('checks/mymodel.meta')
        new_saver.restore(sess,"checks/mymodel")
        
        #注意一个collection相当于一个列表
        W= tf.get_collection("weights")[0]
        print("W:",sess.run(W))
```


### .保存指定的variable

在一个model中可能有很多variable，有时候我们只希望保存其中一部分。我们可以在Saver的构造函数中传入一个var_list属性，只对其中某些列出的variable进行保存。var_list可以是一个字典也可以是一个列表：

- A dict of names to variables: The keys are the names that will be used to save or restore the variables in the checkpoint files. 也就是说，这个key是我们定义的字符串，之后可以用这个key再将key对应的variable

- A list of variables: The variables will be keyed with their op name in the checkpoint files.


```python
v1 = tf.Variable(..., name='v1')
v2 = tf.Variable(..., name='v2')

# Pass the variables as a dict:
saver = tf.train.Saver({'v1': v1, 'v2': v2})

# Or pass them as a list.
saver = tf.train.Saver([v1, v2])
# Passing a list is equivalent to passing a dict with the variable op names
# as keys:
saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})
```